# Antagonism

- This notebook contains code for the hate speech classification model.
- This is a sentiment analaysis problem.
- We will be using the Twitter dataset.
- The term hate speech is understood as any type of verbal, written or behavioural communication that attacks or uses derogatory or discriminatory language against a person or group based on what they are, in other words, based on their religion, ethnicity, nationality, race, colour, ancestry, sex or another identity factor.

### About the dataset
- The data set we will use for the hate speech detection model consists of a test and train set. The training package includes a list of 31,962 tweets, a corresponding ID and a tag 0 or 1 for each tweet. The particular sentiment we need to detect in this dataset is whether or not the tweet is based on hate speech

- 0: NotHate
- 1: Hate

### Importing libraries

In [151]:
import pandas as pd
import re
import pickle

from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score


### Reading the Train and Test dataset 

In [33]:
train_data = pd.read_csv('dataset/train.csv')
test_data = pd.read_csv('dataset/test.csv')


In [34]:
# Reading first 15 rows of the training data
train_data.head(15)


,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
5,6,0,[2/2] huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...
7,8,0,the next school year is the year for exams.ð...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...


In [35]:
# Size of dataset
print("Training Set:"% train_data.columns, train_data.shape) # 31962 rows for Training dataset
print("Testing Set:"% test_data.columns, test_data.shape)    # 171197 rows for Testing dataset


Training Set: (31962, 3)
Testing Set: (17197, 2)


### Cleaning the dataset 

In [36]:
def  clean_text(df, text_field): #
    # This function removes all unnecessary characters from the text
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df

train_clean = clean_text(train_data, "tweet")
test_clean = clean_text(test_data, "tweet")


In [37]:
# You can now see that the tweets are cleaned and readable
train_clean.head(15)


,id,label,tweet
0,1,0,when a father is dysfunctional and is so sel...
1,2,0,thanks for lyft credit i cant use cause they...
2,3,0,bihday your majesty
3,4,0,model i love u take with u all the time in u...
4,5,0,factsguide society now motivation
5,6,0,22 huge fan fare and big talking before they l...
6,7,0,camping tomorrow danny
7,8,0,the next school year is the year for exams can...
8,9,0,we won love the land allin cavs champions clev...
9,10,0,welcome here im its so gr8


In [38]:
# Getting count of Negative and Positive label from train dataset
print("The count for NOT HATE label from train dataset: ", len(train_clean[train_clean.label == 0]))
print("The count for HATE label from train dataset: ", len(train_clean[train_clean.label == 1]))

# The output is clearly imbalanced

The count for NOT HATE label from train dataset:  29720
The count for HATE label from train dataset:  2242


### Handling the imbalance in the data

In [39]:
train_not_hate = train_clean[train_clean.label == 0]
train_hate = train_clean[train_clean.label == 1]

train_hate_upsampled = resample(train_hate, 
                                replace=True,    
                                n_samples=len(train_not_hate),   
                                random_state=123)

train_upsampled = pd.concat([train_hate_upsampled, train_not_hate])
train_upsampled['label'].value_counts()


1    29720
0    29720
Name: label, dtype: int64

### Creating pipeline using sklearn

In [40]:
pipeline_sgd = Pipeline([('vect', CountVectorizer()), 
                         ('tfidf',  TfidfTransformer()), 
                         ('nb', SGDClassifier()),])

### Training the model 

In [41]:
X_train, X_test, y_train, y_test = train_test_split(train_upsampled['tweet'],
                                                    train_upsampled['label'],
                                                    random_state = 0)
model = pipeline_sgd.fit(X_train, y_train)

### Evaluating the model

In [59]:
y_predict = model.predict(X_test)
f1_score_result = f1_score(y_test, y_predict)
accuracy_score_result = accuracy_score(y_test, y_predict)
precision_score_result = precision_score(y_test, y_predict)

print("F1 score %s" % round(f1_score_result*100, 2), "%")
print("Accuracy score %s" % round(accuracy_score_result*100, 2), "%")
print("Precision score %s" % round(precision_score_result*100, 2), "%")

F1 score 96.95 %
Accuracy score 96.92 %
Precision score 95.28 %


### Saving model

In [152]:
pickle.dump(model, open("antagonism_model", 'wb'))

### Loading model

In [153]:
loaded_model = pickle.load(open("antagonism_model", 'rb'))

### Testing the model

In [154]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    return text
    

In [155]:
test_data = pd.read_csv('dataset/test.csv')

for i in range (test_data.size):
    sampleTestTweet = test_data.iloc[i].tweet
    sampleTestTweet = clean_text(sampleTestTweet)

    output = ["Not hate", "hate"]
    result = loaded_model.predict(pd.Series(sampleTestTweet))[0]
    if(result == 1):
        print(sampleTestTweet, "<==>", output[result])
        print()

  white supremacists want everyone to see the new   birds movie  and heres why   <==> hate

thought factory bbc neutrality on right wing fascism  politics media blm brexit trump leadership gt3  <==> hate

 chick gets fucked hottest naked lady  <==> hate

suppo the taiji fisherman no bullying no racism tweet4taiji thecove seashepherd  <==> hate

i say we because im speaking collectively ive always known 2016 showed a lot  and  <==> hate

  trumps invested billions into saudi he empowers the people funding isis   trumpsahypocrite <==> hate

 is there any reason for why the maintainers of the android do not provide more clear and basic code examples in their docs   <==> hate

i believe r willingness 2 try 2 understand 2 encourage amp 2 inspire others 2 b   amp grow is a greater spiritual purpose grok <==> hate

     lt no more feeding at the public trough piggy michelleobama  <==> hate

     your ignorant amp ill informed tweets r silly childish amp one dimensional   <==> hate

hey   a 14

     im not affluent amp there are millions more like meway to just focus on one word smh <==> hate

 why poor vote gop guns religion ignorance aboion fear  <==> hate

   nobdy wants 2 kiss lying conaist   homophobic islamaphobic russianhack nom <==> hate

 so in terms of leg room i was the only one with less space you think that is right   nothappy <==> hate

 allahsoil the temptation to blame everything on a scapegoat is too strong to resist   <==> hate

abrahamhicks  how to bring out the best in yourself and others  via  kindness kind helping help   <==> hate

there is lot of racialhate in america that provides these two lunatics suppo to perpetuate hate   dumptrump   <==> hate

  honestly as long as youre   who the hell cares lifefacts wiz <==> hate

  this belgian team is just a team of talented individuals who have no chemistry whatsoever   <==> hate

 the details of this story are not accurate no evidence for a political act most likely results of mental illness ripjoecox   <==>

  the only way your arguments apply outside of usa is if you think all white countries are the same  <==> hate

just had a company agree to let me go in and do a  information session for all their staff   <==> hate

patriotwatch 411 sherm new year  gap auspol rwnj   drunk refugees asylumseekers politas tas <==> hate

popular youtuber adam saleh ejected from delta flight for speaking arabic   <==> hate

no matter who you are the world will still judge you for your color like these celebs were   <==> hate

trumpusa they call you   homophobic misogynist  jealousy loser     when we go up  they go down  <==> hate

how is it that a man comes in last in a race but he is declared a winner putinschoice putinspuppet retweet  <==> hate

  no one is helpless but we d ppl dont have gutts to come out and stand for our rights and thats really    <==> hate

 why give the orange  bigot bridgeandtunnel wannabe run by retweeting him we all know hes got zero class <==> hate

 cagsil on wizzley   america p

some helpers i refer to as domestic workers because they lack respect how can a black cleaner be so disrespectful to a blackchild   <==> hate

  and tommy the loner is not a muslim  thats what i mean when i said im afraid of politicial campaings  <==> hate

sentimental saturdays may 24 2016 an african in sweden  badinsecret afrosvenskar  <==> hate

thought factory leftright polarisation trump uselections2016 leadership politics  brexit blm 10  <==> hate

a lot of men have a problem with a woman who doesnt need him theyd rather her be desperately dependent so he feels accomplished   <==> hate

  trump celebratingwoman of america a misogynisticabusing fantasyland of equality <==> hate

  not the same for me apparently   <==> hate

 pathetic fallacy infantile  just in case anyone wondered whether  had a brain we have proo  <==> hate

 so you are voting against the fishing communities in favour of a proausterity political union of high youth unemployment   <==> hate

so fucked off  interes

 ends 2016 with a hopeful message against  in the music industry  via  <==> hate

you might be a libtard if libtard  sjw liberal politics  <==> hate

summeime make men more assholes then they already r   <==> hate

  definitely trying this  <==> hate

essential reading by someone you should follow   <==> hate

georgewbush  netanyahu  hate  bombs4votes george amp bb iraq syria two million murder  <==> hate

i am not american but i can clearly see that donald trump will be the worst decision america will ever make should they vote for him   <==> hate

flagging because you might be police  <==> hate

great were going to have a president who subtweets run this country cannot wait to be bombed by other countries for a subtweet    <==> hate

  itwing to play impoant role in 2017 polls list of officebearers    <==> hate

 even if trumps retracts  on all his election rhetorics the dye is cast and the ball of  is set to roll  <==> hate

why social media and the church is making you     <==> hat

...

the us is imploding by its own hand hate bigotry amp intolerance only display weakness solidarity is strength separatism is the enemy   <==> hate

this is pure the only thing u can safely be anti today in canada is white everything else is phobic  <==> hate

so we live in a world of rape enablers because a guy can coach football    <==> hate

  trump doesnt know how timestamps work   <==> hate

is surprised at how racist things have gotten in the us  obama trump  <==> hate

   baroness on marr throwing toys out of pram as folk discuss brexit  and discuss immigration nasty blair <==> hate

 is always looking to make the whiteman look better in comparison to the brownman because of  ideology <==> hate

 it is 2017 and we are still defining  to adults <==> hate

 agree in maldives democracy is purely lip service megalomaniacal imposters abound       <==> hate

 note for those who call wn  for opposing the latest  ad damn right we are get that filth off the n <==> hate

  ive heard in pres

so   i cant order a broth bowl from  because im allergic to soy but they were nice amp checked foodallergy soybeans soup <==> hate

 its also you being covely  look in the mirror brother you are darker than obama keep shucking and jiving for gop <==> hate

 why making the same mistake again if youre first and are fastest do not use aggressive strategies   disappointed <==> hate

  trump the businessman  phony fraud con man trump the man  sexist racist liar trump the politician  gop presidential <==> hate

do people in wheelchairs have sex   sex stories   <==> hate

wow that was the shoest moment of silence ive ever seen and apparently nba fans are a little unclear on the concept of silence   <==> hate

  you and the rest of his family are a bunch of filthy finks nevermypresident lyingtrump crookeddonald r <==> hate

are you black amp feel like the  are stomping on  listen  retweet tampa miami  <==> hate

  is this what you think is ok you hate america as much as the globalists joe    <


 how the supremecou has made it impossible to challenge systemicracism  scotus  kae <==> hate

dear    please define systemicracism and economicapaheid  what do you and or da intend to do to add  <==> hate

 aicle is 10 years oldand nothing changed gotworse   diabetes diabeticretinopathy blindness <==> hate

 black racist fined r450 000  via   southafrica saa <==> hate

just gonna come out and say it  yeah i do hate immigrants  now <==> hate

she has been filtering taxpayer money to her daughters isnt that extoionlock her up and get that money back  <==> hate

tucker sucks  tuckercarlson  <==> hate

treasonoustrump is a fascist  traitor and an existentialthreat to america freedom and the world he mu  <==> hate

 please educate  i am looking for actionable solutions  does trump speak for you trump  active draft dodger  facist <==> hate

ukip kippers happy new year and the oblivion of your  fascist divisivebillionaires people orgnow fraudpaedo liesbest of <==> hate

trump made  comments

proud to say i will represent ireland at2016world university rowing champs in womens senior single scull    <==> hate

 ha good riddance blacklivesmatter  <==> hate

people arent protesting trump because a republican wonthey do so because trump has fuhered  amp  <==> hate

yr spiel says it all right wing anti islam hate eu so much hate    <==> hate

 black girl porn free big tit blonde porn <==> hate

 yep he is the democrats trojan horse   republicans got played   <==> hate

you black amp feel like the  are stomping on you listen  retweet tampa miami  <==> hate

itveuref that rudd woman should think b4 she speaks a statement was asked for all she could do was try and stick the boot in again   <==> hate

   trumpusa when they call us   homophobic misogynist they are really saying they are jealous <==> hate

 office woman cutest girl on eah nude  <==> hate

    to here from the men that brought us  and   <==> hate

 you are a complete jerk your comments are reflections of ignorance and 

IndexError: single positional indexer is out-of-bounds